# release

In [147]:
import torch
import random
from typing import Dict, List
from functools import partial
import datasets
from transformers import (
    AutoTokenizer,
    GPT2Tokenizer,
    GPTNeoXTokenizerFast,
    LlamaTokenizer,
    OPTForCausalLM,
)

In [148]:
class MockArgs:
    raw_dataset_json_path = [
        "/data/tongyx361/reward-by-prm800k/prm800k-main/prm800k/data/phase1_train.jsonl",
        "/data/tongyx361/reward-by-prm800k/prm800k-main/prm800k/data/phase2_train.jsonl",
    ]
    encoded_dataset_output_dirpath = (
        "/data/tongyx361/reward-by-prm800k/datasets/encoded-datasets-pure-prediction"
    )
    preprocessing_num_workers = 16
    overwrite_cache = False
    model_name_or_path = "meta-llama/Llama-2-7b-hf"
    tokenizer_name = "meta-llama/Llama-2-7b-hf"
    use_slow_tokenizer = True
    max_seq_length = None


args = MockArgs()

In [149]:
def pick_prm800k_samples(x):
    return (not x["is_quality_control_question"]) and (
        x["label"]["finish_reason"] in ["found_error", "solution"]
    )


def reformat_prm800k_sample(sample: dict) -> dict:
    problem = sample["question"]["problem"]
    step_ratings = []

    if sample["is_quality_control_question"]:
        raise RuntimeError("is_quality_control_question is True")

    label = sample["label"]
    finish_reason = label["finish_reason"]
    if finish_reason not in ["found_error", "solution"]:
        raise RuntimeError(f"finish_reason is {finish_reason}")

    steps = label["steps"]
    for step in steps:
        chosen_completion = step["chosen_completion"]
        if step["human_completion"] is not None:
            completion = step["human_completion"]
            rating = 1
        else:
            completions = step["completions"]
            if chosen_completion is not None:
                completion = completions[chosen_completion]
            else:
                for completion in completions:
                    if completion["rating"] == -1:
                        break
            rating = completion["rating"]

        step_text = completion["text"]

        if completion["flagged"] not in [None, False]:
            print(f"{sample['timestamp']} flagged: ", completion["flagged"])
            print(sample)
        step_ratings.append({"step": step_text, "rating": rating})

    reformatted_sample = {"problem": problem, "step_ratings": step_ratings}

    if finish_reason == "found_error":
        last_rating = reformatted_sample["step_ratings"][-1]["rating"]
        assert last_rating == -1, f"last step should be -1 but {last_rating}"

    return reformatted_sample


def encode_with_problem_step_ratings_format(reformatted_sample, tokenizer, test=False):
    """
    Here we assume each sample has a 'step_ratings' field. Each step_rating is a dict.
    """

    step_ratings = reformatted_sample["step_ratings"]
    if len(step_ratings) == 0:
        raise ValueError("step_ratings field is empty.")

    rating2word = {1: "positive", -1: "negative", 0: "neutral"}
    rating2token_id = {
        rating: tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))[0]
        for rating, word in rating2word.items()
    }

    problem = reformatted_sample["problem"].strip()
    problem_step_ratings_text = problem + "\n"
    sample_input_ids = tokenizer(
        problem + "\n",
        return_tensors="pt",
        padding=False,
        truncation=False,
        add_special_tokens=True,
        return_attention_mask=False,
    )["input_ids"]
    ignore_index = -100
    sample_labels = torch.ones_like(sample_input_ids) * ignore_index

    for step_rating in step_ratings:
        step = step_rating["step"].strip()

        problem_step_ratings_text += step + "\n"

        step_input_ids = tokenizer(
            "\n" + step + "\n",
            return_tensors="pt",
            padding=False,
            truncation=False,
            # add_special_tokens=True,
            add_special_tokens=False,
            return_attention_mask=False,
        )["input_ids"]
        step_input_ids = step_input_ids[:, 2:]  # remove "\n"
        sample_input_ids = torch.cat((sample_input_ids, step_input_ids), dim=1)

        step_labels = torch.ones_like(step_input_ids) * ignore_index
        step_labels[:, -1] = rating2token_id[
            step_rating["rating"]
        ]  # set the label for the last token_id before "\n"
        sample_labels = torch.cat((sample_labels, step_labels), dim=1)

    # keep the last token for hugging face to align input_ids and labels

    if test:
        sample_input_ids_from_simple_call = tokenizer(
            problem_step_ratings_text,
            return_tensors="pt",
            padding=False,
            truncation=False,
            add_special_tokens=True,
            return_attention_mask=False,
        )["input_ids"]
        assert torch.equal(sample_input_ids, sample_input_ids_from_simple_call), (
            sample_input_ids != sample_input_ids_from_simple_call
        )

    attention_mask = torch.ones_like(sample_input_ids)
    train_sample = {
        "input_ids": sample_input_ids.flatten(),
        "labels": sample_labels.flatten(),
        "attention_mask": attention_mask.flatten(),
    }
    return train_sample

In [150]:
dataset_args = {
    # "streaming": True,
}

reformat_col_names = ["problem", "step_ratings"]
train_data_files_list = args.raw_dataset_json_path
reformatted_prm800k_phase_train_list = [None] * len(train_data_files_list)
for idx, data_files_train in enumerate(train_data_files_list):
    prm800k_phase_train = datasets.load_dataset(
        "json",
        data_files={"train": data_files_train},
        split="train",
    )
    filtered_prm800k_phase_train = prm800k_phase_train.filter(pick_prm800k_samples)
    reformatted_filtered_prm800k_phase_train = filtered_prm800k_phase_train.map(
        reformat_prm800k_sample,
        batched=False,
        num_proc=args.preprocessing_num_workers,
        load_from_cache_file=not args.overwrite_cache,
        remove_columns=[
            name
            for name in filtered_prm800k_phase_train.column_names
            if name not in reformat_col_names
        ],
        desc="Reformatting(extractting) SFT data",
    )
    reformatted_prm800k_phase_train_list[idx] = reformatted_filtered_prm800k_phase_train
train_dataset = datasets.concatenate_datasets(reformatted_prm800k_phase_train_list)
raw_datasets = datasets.DatasetDict(
    {
        "train": train_dataset,
    }
)

Found cached dataset json (/data/tongyx361/.cache/huggingface/datasets/json/default-2a52b90631375b50/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
Loading cached processed dataset at /data/tongyx361/.cache/huggingface/datasets/json/default-2a52b90631375b50/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-135f7396c4187c64.arrow
Loading cached processed dataset at /data/tongyx361/.cache/huggingface/datasets/json/default-2a52b90631375b50/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-2a20514ec136c5e8_*_of_00016.arrow
Found cached dataset json (/data/tongyx361/.cache/huggingface/datasets/json/default-0f474ed7b76bb2f2/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
Loading cached processed dataset at /data/tongyx361/.cache/huggingface/datasets/json/default-0f474ed7b76bb2f2/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-a7c500be6ba24b6c.arrow
Loading cache

In [153]:
# import os

# os.environ["TRANSFORMERS_OFFLINE"] = "1"

model = None

tokenizer = AutoTokenizer.from_pretrained(
    "/data/tongyx361/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9"
)

# tokenizer
# if args.tokenizer_name:
#     tokenizer = AutoTokenizer.from_pretrained(
#         args.tokenizer_name, use_fast=not args.use_slow_tokenizer
#     )
# elif args.model_name_or_path:
#     tokenizer = AutoTokenizer.from_pretrained(
#         args.model_name_or_path, use_fast=not args.use_slow_tokenizer
#     )
# else:
#     raise ValueError(
#         "You are instantiating a new tokenizer from scratch. This is not supported by this script."
#         "You can do it from another script, save it, and load it from here, using --tokenizer_name."
#     )

# no default pad token for llama!
# here we add all special tokens again, because the default ones are not in the special_tokens_map
if isinstance(tokenizer, LlamaTokenizer):
    num_added_tokens = tokenizer.add_special_tokens(
        {
            "bos_token": "<s>",
            "eos_token": "</s>",
            "unk_token": "<unk>",
            "pad_token": "<pad>",
        }
    )
    assert num_added_tokens in [
        0,
        1,
    ], "LlamaTokenizer should only add one special token - the pad_token, or no tokens if pad token present."
elif isinstance(tokenizer, GPTNeoXTokenizerFast):
    num_added_tokens = tokenizer.add_special_tokens(
        {
            "pad_token": "<pad>",
        }
    )
    assert (
        num_added_tokens == 1
    ), "GPTNeoXTokenizer should only add one special token - the pad_token."
elif isinstance(tokenizer, GPT2Tokenizer) and isinstance(model, OPTForCausalLM):
    num_added_tokens = tokenizer.add_special_tokens({"unk_token": "<unk>"})
encode_function = partial(
    encode_with_problem_step_ratings_format,
    tokenizer=tokenizer,
)

In [154]:
lm_datasets = raw_datasets.map(
    encode_function,
    batched=False,
    num_proc=args.preprocessing_num_workers,
    load_from_cache_file=not args.overwrite_cache,
    remove_columns=[
        name
        for name in raw_datasets["train"].column_names
        if name not in ["input_ids", "labels", "attention_mask"]
    ],
    desc="Tokenizing and reformatting SFT data",
)
lm_datasets.set_format(type="pt")
lm_datasets = lm_datasets.filter(lambda example: (example["labels"] != -100).any())


train_dataset = lm_datasets["train"]

Tokenizing and reformatting SFT data (num_proc=16):   0%|          | 0/87012 [00:00<?, ? examples/s]

Filter:   0%|          | 0/87012 [00:00<?, ? examples/s]

In [157]:
lm_datasets.save_to_disk(args.encoded_dataset_output_dirpath)

Saving the dataset (0/1 shards):   0%|          | 0/87012 [00:00<?, ? examples/s]

# test

In [155]:
print(raw_datasets["train"][random.randint(0, len(raw_datasets["train"]))])

{'problem': 'If $a$ and $b$ are positive integers for which $ab - 6a + 5b = 373$, what is the minimal possible value of $|a - b|$?', 'step_ratings': [{'rating': -1, 'step': 'The first thing I notice is that the equation is symmetric in $a$ and $b$, meaning that if I swap $a$ and $b$, the equation stays the same.'}]}


In [156]:
print(len(train_dataset))  # 87012
print(train_dataset[random.randint(0, len(train_dataset))])

87012
{'input_ids': tensor([    1,  1724,   338,   278, 14176,  3619,  8572,   272,   310, 29871,
        29896, 29946, 29900, 29955,   322, 29871, 29929, 29900, 29941, 29973,
           13,  1762,  1284,   278, 14176,  3619,  8572,   272,   313, 29954,
         6530, 29897,   310,  1023,  3694, 29892,   306,   817,   304,  1284,
          278, 10150,  6374,  6043,   393,  1933,  2247,  1716,  3694,  1584,
          368, 29889,    13,  6716,   982,   304,   437,   445,   338,   304,
          671,   278,   382, 27511,  5687, 29892,   607,  4083,   393,   278,
          402,  6530,   310,  1023,  3694,   263,   322,   289,   338,   278,
         1021,   408,   278,   402,  6530,   310,   289,   322,   278, 21162,
          310,   263, 13931,   491,   289, 29889,    13,  6295, 29892,   306,
        29915,   645,  1369,   491,  1933,  4821, 29871, 29896, 29946, 29900,
        29955,   491, 29871, 29929, 29900, 29941,   322,  9138,   278, 21162,
        29889,    13, 15156,  1472,  8542, 2

# debug

In [ ]:
# raw_dataset_sample = raw_dataset[random.randint(0, len(raw_dataset))]
# print(raw_dataset_sample)
raw_dataset_example = {
    "labeler": "68e6d5f3-747c-4420-97b4-a30a73f639f9",
    "timestamp": "2022-11-01T17:20:23.427734",
    "generation": 4,
    "is_quality_control_question": False,
    "is_initial_screening_question": False,
    "question": {
        "problem": "If the system of equations\n\n\\begin{align*}\n6x-4y&=a,\\\\\n6y-9x &=b.\n\\end{align*}has a solution $(x, y)$ where $x$ and $y$ are both nonzero, find $\\frac{a}{b},$ assuming $b$ is nonzero.",
        "ground_truth_solution": "If we multiply the first equation by $-\\frac{3}{2}$, we obtain\n\n$$6y-9x=-\\frac{3}{2}a.$$Since we also know that $6y-9x=b$, we have\n\n$$-\\frac{3}{2}a=b\\Rightarrow\\frac{a}{b}=\\boxed{-\\frac{2}{3}}.$$",
        "ground_truth_answer": "-\\frac{2}{3}",
        "pre_generated_steps": [
            "I notice that the coefficients of the system are multiples of 3, so I can divide both equations by 3 to simplify them.",
            "\\begin{align*}\n2x-\\frac{4}{3}y&=\\frac{a}{3},\\\\\n2y-3x &=\\frac{b}{3}.\n\\end{align*}",
            "I also notice that the coefficients of $x$ and $y$ in the second equation are the negatives of the coefficients of $y$ and $x$ in the first equation, respectively.",
            "This suggests that I can add the two equations to eliminate one of the variables.",
            "\\begin{align*}\n2x-\\frac{4}{3}y+\\left(2y-3x\\right)&=\\frac{a}{3}+\\frac{b}{3},\\\\\n\\frac{2}{3}y- x &=\\frac{a+b}{3}.\n\\end{align*}",
            "Now I can solve for $y$ in terms of $x$ and the constants.",
            "\\begin{align*}\n\\frac{2}{3}y&= x + \\frac{a+b}{3},\\\\\ny&= \\frac{3}{2}x + \\frac{a+b}{2}.\n\\end{align*}",
            "I can substitute this expression for $y$ into either of the original equations to get an equation involving only $x$ and the constants.",
            "I choose the first equation for convenience.",
            "\\begin{align*}\n2x-\\frac{4}{3}\\left(\\frac{3}{2}x + \\frac{a+b}{2}\\right)&=\\frac{a}{3},\\\\\n2x-2x - \\frac{2}{3}\\left(a+b\\right)&=\\frac{a}{3},\\\\\n-\\frac{2}{3}\\left(a+b\\right)&=\\frac{a}{3},\\\\\n-2\\left(a+b\\right)&=a,\\\\\na&=-2b.\n\\end{align*}",
            "I have found a relationship between $a$ and $b$ that must hold for the system to have a nonzero solution.",
            "I can use this to find the ratio $\\frac{a}{b}$ by dividing both sides by $b$.",
            "\\begin{align*}\n\\frac{a}{b}&=\\frac{-2b}{b},\\\\\n\\frac{a}{b}&=-2.\n\\end{align*}",
            "This is the answer I was looking for.",
            "# Answer\n\n-2",
        ],
        "pre_generated_answer": "-2",
        "pre_generated_verifier_score": 0.12514245681574954,
    },
    "label": {
        "steps": [
            {
                "completions": [
                    {
                        "text": "I notice that the coefficients of the system are multiples of 3, so I can divide both equations by 3 to simplify them.",
                        "rating": 0,
                        "flagged": None,
                    }
                ],
                "human_completion": None,
                "chosen_completion": 0,
            },
            {
                "completions": [
                    {
                        "text": "\\begin{align*}\n2x-\\frac{4}{3}y&=\\frac{a}{3},\\\\\n2y-3x &=\\frac{b}{3}.\n\\end{align*}",
                        "rating": 1,
                        "flagged": None,
                    }
                ],
                "human_completion": None,
                "chosen_completion": 0,
            },
            {
                "completions": [
                    {
                        "text": "I also notice that the coefficients of $x$ and $y$ in the second equation are the negatives of the coefficients of $y$ and $x$ in the first equation, respectively.",
                        "rating": -1,
                        "flagged": None,
                    },
                    {
                        "text": "To eliminate one of the variables, I can try to make the coefficients of $x$ or $y$ match in both equations. I see that multiplying the first equation by $\\frac{3}{4}$ and the second equation by $-\\frac{4}{3}$ will achieve this for $y$.",
                        "rating": -1,
                        "flagged": False,
                    },
                    {
                        "text": "I want to eliminate one of the variables, so I multiply the first equation by $\\frac{3}{2}$ and the second equation by $x$ to get",
                        "rating": -1,
                        "flagged": False,
                    },
                    {
                        "text": "I want to find a way to eliminate one of the variables and solve for the other, so I can use the substitution or elimination method.",
                        "rating": 1,
                        "flagged": False,
                    },
                    {
                        "text": "I want to eliminate one of the variables, so I can multiply the first equation by $\\frac{3}{2}$ and add it to the second equation.",
                        "rating": -1,
                        "flagged": False,
                    },
                ],
                "human_completion": None,
                "chosen_completion": None,
            },
        ],
        "total_time": 418405,
        "finish_reason": "found_error",
    },
}

In [18]:
# test
reformatted_raw_dataset_example = reformat_prm800k_sample(raw_dataset_example)
print(reformatted_raw_dataset_example)

{'problem': 'If the system of equations\n\n\\begin{align*}\n6x-4y&=a,\\\\\n6y-9x &=b.\n\\end{align*}has a solution $(x, y)$ where $x$ and $y$ are both nonzero, find $\\frac{a}{b},$ assuming $b$ is nonzero.', 'step_ratings': [{'step': 'I notice that the coefficients of the system are multiples of 3, so I can divide both equations by 3 to simplify them.', 'rating': 0}, {'step': '\\begin{align*}\n2x-\\frac{4}{3}y&=\\frac{a}{3},\\\\\n2y-3x &=\\frac{b}{3}.\n\\end{align*}', 'rating': 1}, {'step': 'I also notice that the coefficients of $x$ and $y$ in the second equation are the negatives of the coefficients of $y$ and $x$ in the first equation, respectively.', 'rating': -1}]}


In [14]:
# debug
newline_id = tokenizer.convert_tokens_to_ids("\n")
print(newline_id)
newline_in_sentence_ids = tokenizer.encode("a\nb")
print(newline_in_sentence_ids)

0
[1, 263, 13, 29890]


In [119]:
test = True
test_for = {"token_prefix": True, "equal": True, "common": False}

In [91]:
reformatted_sample = reformatted_raw_dataset_example

# function
if test_for == True:
    print("reformatted_sample: ", reformatted_sample, sep="\n", end="\n\n")
step_ratings = reformatted_sample["step_ratings"]
if len(step_ratings) == 0:
    raise ValueError("step_ratings field is empty.")

rating2word = {1: "positive", -1: "negative", 0: "neutral"}
rating2token_id = {
    rating: tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))[0]
    for rating, word in rating2word.items()
}

problem = reformatted_sample["problem"].strip()
problem_step_ratings_text = problem + "\n"
sample_input_ids = tokenizer(
    problem + "\n",
    return_tensors="pt",
    padding=False,
    truncation=False,
    add_special_tokens=True,
    return_attention_mask=False,
)["input_ids"]
ignore_index = -100
sample_labels = torch.ones_like(sample_input_ids) * ignore_index

for step_rating in step_ratings:
    step = step_rating["step"].strip()

    problem_step_ratings_text += step + "\n"

    step_input_ids = tokenizer(
        "\n" + step + "\n",
        return_tensors="pt",
        padding=False,
        truncation=False,
        # add_special_tokens=True,
        add_special_tokens=False,
        return_attention_mask=False,
    )["input_ids"]
    step_input_ids = step_input_ids[:, 2:]  # remove "\n"
    sample_input_ids = torch.cat((sample_input_ids, step_input_ids), dim=1)

    step_labels = torch.ones_like(step_input_ids) * ignore_index
    step_labels[:, -2] = rating2token_id[
        step_rating["rating"]
    ]  # set the label for the last token_id before "\n"
    sample_labels = torch.cat((sample_labels, step_labels), dim=1)

sample_input_ids = sample_input_ids[:, :-1]
sample_labels = sample_labels[:, :-1]
problem_step_ratings_text = problem_step_ratings_text.strip()

if test:
    sample_input_ids_from_simple_call = tokenizer(
        problem_step_ratings_text,
        return_tensors="pt",
        padding=False,
        truncation=False,
        add_special_tokens=True,
        return_attention_mask=False,
    )["input_ids"]
    assert torch.equal(sample_input_ids, sample_input_ids_from_simple_call), (
        sample_input_ids != sample_input_ids_from_simple_call
    )

attention_mask = torch.ones_like(sample_input_ids)
train_sample = {
    "input_ids": sample_input_ids.flatten(),
    "labels": sample_labels.flatten(),
    "attention_mask": attention_mask.flatten(),
}

reformatted_sample: 
{'problem': 'If the system of equations\n\n\\begin{align*}\n6x-4y&=a,\\\\\n6y-9x &=b.\n\\end{align*}has a solution $(x, y)$ where $x$ and $y$ are both nonzero, find $\\frac{a}{b},$ assuming $b$ is nonzero.', 'step_ratings': [{'step': 'I notice that the coefficients of the system are multiples of 3, so I can divide both equations by 3 to simplify them.', 'rating': 0}, {'step': '\\begin{align*}\n2x-\\frac{4}{3}y&=\\frac{a}{3},\\\\\n2y-3x &=\\frac{b}{3}.\n\\end{align*}', 'rating': 1}, {'step': 'I also notice that the coefficients of $x$ and $y$ in the second equation are the negatives of the coefficients of $y$ and $x$ in the first equation, respectively.', 'rating': -1}]}

sample_text: 
If the system of equations

\begin{align*}
6x-4y&=a,\\
6y-9x &=b.
\end{align*}has a solution $(x, y)$ where $x$ and $y$ are both nonzero, find $\frac{a}{b},$ assuming $b$ is nonzero.
I notice that the coefficients of the system are multiples of 3, so I can divide both equations by 3 t

AttributeError: 'LlamaTokenizer' object has no attribute 'char_to_token'

In [121]:
# debug
def simple_text2token_ids(tokenizer, text, **kwargs):
    return tokenizer(
        text, return_tensors="pt", padding=False, truncation=False, **kwargs
    )["input_ids"]


def print_decoded_tokens(input_ids):
    decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    print(
        "decoded_tokens:",
        decoded_tokens,
        sep="\n",
        end="\n\n",
    )


def print_tokenization(tokenizer, text):
    # print(
    #     f'text:\n"""{text}"""',
    #     end="\n\n",
    # )
    # text_prepared_for_tokenization = tokenizer.prepare_for_tokenization(text)
    # print(
    #     "text_prepared_for_tokenizations:",
    #     text_prepared_for_tokenization,
    #     sep="\n",
    #     end="\n\n",
    # )
    # tokens = tokenizer.tokenize(text, **text_prepared_for_tokenization[-1])
    # print(
    #     "tokens:",
    #     tokens,
    #     sep="\n",
    #     end="\n\n",
    # )
    # input_ids = tokenizer.convert_tokens_to_ids(tokens)
    # print(
    #     "input_ids:",
    #     input_ids,
    #     sep="\n",
    #     end="\n\n",
    # )
    # input_ids_from_call_wo_special_tokens = simple_text2token_ids(
    #     tokenizer, text, add_special_tokens=False
    # )
    # print(
    #     "input_ids_from_call_wo_special_tokens:",
    #     input_ids_from_call_wo_special_tokens[0],
    #     sep="\n",
    #     end="\n\n",
    # )
    input_ids_from_simple_call = simple_text2token_ids(tokenizer, text)
    print(
        "input_ids_from_simple_call:",
        input_ids_from_simple_call[0],
        sep="\n",
        end="\n\n",
    )

    print_decoded_tokens(input_ids_from_simple_call)


rating2word = {1: "positive", -1: "negative", 0: "neutral"}
rating2token_id = {
    rating: tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))[0]
    for rating, word in rating2word.items()
}

problem = reformatted_sample["problem"].strip()
problem_step_ratings_text = problem + "\n"
sample_input_ids = tokenizer(
    problem + "\n",
    return_tensors="pt",
    padding=False,
    truncation=False,
    add_special_tokens=True,
    return_attention_mask=False,
)["input_ids"]
ignore_index = -100
sample_labels = torch.ones_like(sample_input_ids) * ignore_index

if test:
    if test_for["common"]:
        print_tokenization(tokenizer, problem_step_ratings_text)
        print(
            "sample_labels:",
            sample_labels,
            sep="\n",
            end="\n\n",
        )


for step_rating in step_ratings:
    step = step_rating["step"].strip()

    problem_step_ratings_text += step + "\n"

    step_input_ids = tokenizer(
        "\n" + step + "\n",
        return_tensors="pt",
        padding=False,
        truncation=False,
        # add_special_tokens=True,
        add_special_tokens=False,
        return_attention_mask=False,
    )["input_ids"]
    step_input_ids = step_input_ids[:, 2:]  # remove "\n"
    sample_input_ids = torch.cat((sample_input_ids, step_input_ids), dim=1)

    step_labels = torch.ones_like(step_input_ids) * ignore_index
    step_labels[:, -2] = rating2token_id[
        step_rating["rating"]
    ]  # set the label for the last token_id before "\n"
    sample_labels = torch.cat((sample_labels, step_labels), dim=1)

    if test:
        if test_for["common"]:
            print_tokenization(tokenizer, problem_step_ratings_text)
            print(
                "sample_labels:",
                sample_labels,
                sep="\n",
                end="\n\n",
            )
        if test_for["token_prefix"]:
            print_decoded_tokens(step_input_ids)


sample_input_ids = sample_input_ids[:, :-1]
sample_labels = sample_labels[:, :-1]
problem_step_ratings_text = problem_step_ratings_text.strip()

sample_input_ids_from_simple_call = tokenizer(
    problem_step_ratings_text,
    return_tensors="pt",
    padding=False,
    truncation=False,
    add_special_tokens=True,
    return_attention_mask=False,
)["input_ids"]


if test:
    if test_for["token_prefix"]:
        sample_input_ids_from_simple_call = tokenizer(
            problem_step_ratings_text,
            return_tensors="pt",
            padding=False,
            truncation=False,
            add_special_tokens=True,
            return_attention_mask=False,
        )["input_ids"]
        print_decoded_tokens(sample_input_ids_from_simple_call)
    if test_for["common"]:
        print(
            "sample_input_ids:",
            sample_input_ids,
            sep="\n",
            end="\n\n",
        )
        print(
            "sample_labels:",
            sample_labels,
            sep="\n",
            end="\n\n",
        )
    if test_for["equal"]:
        print(
            "sample_input_ids_from_simple_call:",
            sample_input_ids_from_simple_call,
            sep="\n",
            end="\n\n",
        )

        def print_difference(input_ids, different_mask):
            difference = input_ids * different_mask
            print(difference)
            print(tokenizer.convert_ids_to_tokens(difference[0]))

        def print_difference_between(input_ids_pair):
            different_mask = input_ids_pair[0] != input_ids_pair[1]
            for input_ids in input_ids_pair:
                print_difference(input_ids, different_mask)

        print_difference_between((sample_input_ids, sample_input_ids_from_simple_call))

        assert torch.equal(sample_input_ids, sample_input_ids_from_simple_call)

decoded_tokens:
['I', '▁notice', '▁that', '▁the', '▁coefficients', '▁of', '▁the', '▁system', '▁are', '▁multi', 'ples', '▁of', '▁', '3', ',', '▁so', '▁I', '▁can', '▁divide', '▁both', '▁equations', '▁by', '▁', '3', '▁to', '▁simplify', '▁them', '.', '<0x0A>']

decoded_tokens:
['\\', 'begin', '{', 'align', '*}', '<0x0A>', '2', 'x', '-\\', 'frac', '{', '4', '}{', '3', '}', 'y', '&', '=\\', 'frac', '{', 'a', '}{', '3', '},', '\\\\', '<0x0A>', '2', 'y', '-', '3', 'x', '▁&=\\', 'frac', '{', 'b', '}{', '3', '}.', '<0x0A>', '\\', 'end', '{', 'align', '*}', '<0x0A>']

decoded_tokens:
['I', '▁also', '▁notice', '▁that', '▁the', '▁coefficients', '▁of', '▁$', 'x', '$', '▁and', '▁$', 'y', '$', '▁in', '▁the', '▁second', '▁equation', '▁are', '▁the', '▁neg', 'atives', '▁of', '▁the', '▁coefficients', '▁of', '▁$', 'y', '$', '▁and', '▁$', 'x', '$', '▁in', '▁the', '▁first', '▁equation', ',', '▁respectively', '.', '<0x0A>']

decoded_tokens:
['<s>', '▁If', '▁the', '▁system', '▁of', '▁equations', '<0x0A>', '<0x

In [88]:
# debug: rating tokens
for rating_token in rating2token.values():
    print(tokenizer.tokenize(rating_token))
    print(tokenizer.convert_tokens_to_ids(rating_token))

['▁positive']
0
['▁negative']
22198
['▁neutral']
0


In [78]:
# debug: return
tokenizer(
    problem_step_ratings_text,
    return_tensors="pt",
    padding=False,
    truncation=False,
    add_special_tokens=True,
    return_attention_mask=False,
)

{'input_ids': tensor([[    1,   960,   278,  1788,   310, 10693,    13,    13, 29905,   463,
         29912,  2520,  4044,    13, 29953, 29916, 29899, 29946, 29891, 20644,
         29874, 29892,  1966,    13, 29953, 29891, 29899, 29929, 29916,  7878,
         29890, 29889,    13, 29905,   355, 29912,  2520,  4044,  5349,   263,
          1650,  2427, 29916, 29892,   343,  1262,   988,   395, 29916, 29938,
           322,   395, 29891, 29938,   526,  1716,  1661,  9171, 29892,  1284,
           779,  1154, 29912, 29874,  1157, 29890,  1118, 29938, 10241,   395,
         29890, 29938,   338,  1661,  9171, 29889,    13, 29902,  8369,   393,
           278, 16127,   310,   278,  1788,   526,  2473,  2701,   310, 29871,
         29941, 29892,   577,   306,   508, 16429,  1716, 10693,   491, 29871,
         29941,   304, 21092,   963, 29889,    13, 29905,   463, 29912,  2520,
          4044,    13, 29906, 29916,  2612,  1154, 29912, 29946,  1157, 29941,
         29913, 29891, 29987,  2013,  

In [64]:
# debug

print(
    tokenizer.tokenize(
        "If the system of equations\n\n\\begin{align*}\n6x-4y&=a,\\\\\n6y-9x &=b.\n\\end{align*}has a solution $(x, y)$ where $x$ and $y$ are both nonzero, find $\\frac{a}{b},$ assuming $b$ is nonzero."
    )
)

print(tokenizer.tokenize("rating"))
print(tokenizer.tokenize("something\n"))
print(tokenizer.tokenize("something\n")[-1])  # ['▁something', '<0x0A>']
print(tokenizer("something\n")[] [-1])  # ['▁something', '<0x0A>']

['▁If', '▁the', '▁system', '▁of', '▁equations', '<0x0A>', '<0x0A>', '\\', 'begin', '{', 'align', '*}', '<0x0A>', '6', 'x', '-', '4', 'y', '&=', 'a', ',', '\\\\', '<0x0A>', '6', 'y', '-', '9', 'x', '▁&=', 'b', '.', '<0x0A>', '\\', 'end', '{', 'align', '*}', 'has', '▁a', '▁solution', '▁$(', 'x', ',', '▁y', ')$', '▁where', '▁$', 'x', '$', '▁and', '▁$', 'y', '$', '▁are', '▁both', '▁non', 'zero', ',', '▁find', '▁$\\', 'frac', '{', 'a', '}{', 'b', '},', '$', '▁assuming', '▁$', 'b', '$', '▁is', '▁non', 'zero', '.']
['▁rating']
['▁something', '<0x0A>']
<0x0A>


KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'

In [69]:
# debug: whether concatenated token_ids equals to the original token_ids


problem = reformatted_sample["problem"].strip()
problem_step_ratings_text = problem + "\n"
sample_input_ids = simple_text2token_ids(tokenizer, problem_step_ratings_text)
sample_rating_idx = []

if test:
    print_tokenization(tokenizer, problem_step_ratings_text)

for step_rating in step_ratings:
    step = step_rating["step"].strip()
    step

    problem_step_ratings_text += step + "\n"


sample_input_ids = simple_text2token_ids(tokenizer, problem_step_ratings_text)
if test:
    print_tokenization(tokenizer, problem_step_ratings_text)

text:
"""If the system of equations

\begin{align*}
6x-4y&=a,\\
6y-9x &=b.
\end{align*}has a solution $(x, y)$ where $x$ and $y$ are both nonzero, find $\frac{a}{b},$ assuming $b$ is nonzero.
"""

input_ids:
tensor([    1,   960,   278,  1788,   310, 10693,    13,    13, 29905,   463,
        29912,  2520,  4044,    13, 29953, 29916, 29899, 29946, 29891, 20644,
        29874, 29892,  1966,    13, 29953, 29891, 29899, 29929, 29916,  7878,
        29890, 29889,    13, 29905,   355, 29912,  2520,  4044,  5349,   263,
         1650,  2427, 29916, 29892,   343,  1262,   988,   395, 29916, 29938,
          322,   395, 29891, 29938,   526,  1716,  1661,  9171, 29892,  1284,
          779,  1154, 29912, 29874,  1157, 29890,  1118, 29938, 10241,   395,
        29890, 29938,   338,  1661,  9171, 29889,    13])

tokens:
['<s>', '▁If', '▁the', '▁system', '▁of', '▁equations', '<0x0A>', '<0x0A>', '\\', 'begin', '{', 'align', '*}', '<0x0A>', '6', 'x', '-', '4', 'y', '&=', 'a', ',', '\\\\', '<0x0A>', 

In [58]:
# dehug: whether newline at the beginning will be removed
text = "a"
text_in_newline = "\n" + text + "\n" + text + "\n\n" + text + "\n"
sample_input_ids = simple_text2token_ids(tokenizer, text_in_newline)
sample_rating_idx = []

if test:
    print_tokenization(text_in_newline, sample_input_ids)

text:
"""
a
a

a
"""

input_ids:
tensor([    1, 29871,    13, 29874,    13, 29874,    13,    13, 29874,    13])

tokens:
['<s>', '▁', '<0x0A>', 'a', '<0x0A>', 'a', '<0x0A>', '<0x0A>', 'a', '<0x0A>']



In [ ]:
# test

In [ ]:
def prepare_dataset_for_collator(raw_dataset_json_path):
    # load
    # if list
    
    
    # filter
    
    # reformat
    
    # encode
    
    # sample check